# Setup & Keys

In [1]:
import json

# Load the secret keys from the JSON file
def load_secret_keys(file_path):
    with open(file_path, 'r') as file:
        secret_keys = json.load(file)
    return secret_keys
keys =load_secret_keys('secrets.json')

In [2]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = keys['Groq']
os.environ["GROQ_API_KEY"] = keys['Groq']

# Language Model

In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Pawan")])

AIMessage(content='Nice to meet you, Pawan! Is there something I can help you with or would you like to chat?', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 16, 'total_tokens': 40, 'completion_time': 0.018736656, 'prompt_time': 0.003637981, 'queue_time': None, 'total_time': 0.022374637000000003}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-b737f47a-2839-40f8-b003-98ae52d71a31-0', usage_metadata={'input_tokens': 16, 'output_tokens': 24, 'total_tokens': 40})

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch 

In [5]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm happy to help! However, I'm a large language model, I don't have the ability to know your name unless you've told me or provided it to me. I'm a new conversation every time you interact with me, so I don't retain any information from previous conversations. If you'd like to share your name with me, I'd be happy to learn it!", response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 15, 'total_tokens': 94, 'completion_time': 0.063835788, 'prompt_time': 0.003484142, 'queue_time': None, 'total_time': 0.06731993}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-11114225-b287-43b9-b09b-fdf3103d1d4d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 79, 'total_tokens': 94})

In [6]:
from langchain_core.messages import AIMessage

response = model.invoke(
    [
        HumanMessage(content="Hi! I'm Pawan"),
        AIMessage(content="Hello Pawan! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)
print('RESPONSE CONTENT -->>',response.content)
print('RESPONSE METADATA -->>',response.response_metadata)
print('RESPONSE ID -->>',response.id)
print('RESPONSE USAGE METADATA -->>',response.usage_metadata)
response

RESPONSE CONTENT -->> Your name is Pawan!
RESPONSE METADATA -->> {'token_usage': {'completion_tokens': 7, 'prompt_tokens': 42, 'total_tokens': 49, 'completion_time': 0.004754409, 'prompt_time': 0.00694623, 'queue_time': None, 'total_time': 0.011700639}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}
RESPONSE ID -->> run-b92a235f-38eb-4677-9f23-4be968447d1f-0
RESPONSE USAGE METADATA -->> {'input_tokens': 42, 'output_tokens': 7, 'total_tokens': 49}


AIMessage(content='Your name is Pawan!', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 42, 'total_tokens': 49, 'completion_time': 0.004754409, 'prompt_time': 0.00694623, 'queue_time': None, 'total_time': 0.011700639}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-b92a235f-38eb-4677-9f23-4be968447d1f-0', usage_metadata={'input_tokens': 42, 'output_tokens': 7, 'total_tokens': 49})

# Message History

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [8]:
config = {"configurable": {"session_id": "abc2"}}

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Pawan")],
    config=config,
)

response.content

'Hello Pawan! Nice to meet you! Is there something I can help you with or would you like to chat?'

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Pawan!'

In [11]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm happy to help! However, I'm a large language model, I don't have the ability to know your personal information, including your name. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to share your name with me, I'd be happy to chat with you and get to know you better!"

In [12]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'I already told you! Your name is Pawan!'

# Prompt templates

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [14]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm Pawan")]})

response.content

"Hello Pawan! Nice to meet you! I'm here to help you with any questions or topics you'd like to discuss. What's on your mind today?"

In [15]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [16]:
config = {"configurable": {"session_id": "abc5"}}

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Gunjan")],
    config=config,
)

response.content

"Hello Gunjan! It's nice to meet you. I'm here to help you with any questions or topics you'd like to discuss. How's your day going so far?"

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm glad you asked! Your name is Gunjan."

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [20]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm Pawan")], "language": "English"}
)

response.content

'Hi Pawan! Nice to meet you! How can I assist you today?'

In [21]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [22]:
config = {"configurable": {"session_id": "abc11"}}

In [23]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm Gunajn")], "language": "English"},
    config=config,
)

response.content

"Hello Gunajn! It's nice to meet you! I'm here to help you with any questions or topics you'd like to discuss. What's on your mind today?"

In [24]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "English"},
    config=config,
)

response.content

'You told me your name is Gunajn!'

# Managing Conversation History

In [25]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Pawan"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like Samosa"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm Pawan"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like Samosa'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [26]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

'I remember! Your name is Pawan!'

In [27]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked: "whats 2 + 2"'

In [28]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [29]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Pawan!'

In [30]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked me "whats 2 + 2"!'

# Streaming

In [31]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

|Hi| Todd|!| Here|'s| one|:

|Why| couldn|'t| the| bicycle| stand| up| by| itself|?

|(W|ait| for| it|...)

|Because| it| was| two|-t|ired|!

|Hope| that| made| you| laugh|!||